## RankForecasting-stint-paper

baseed on: 18/stint_test

stint rank prediction for paper



In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    
    #the simplest case, trim=0
    stint._trim = 0
    #stint._debug_carlist=[6]
    #stint._debug_carlist=[59]
    stint._debug_carlist=[]
    
    stint._force_endpit_align = False
    stint._include_endpit = True
    
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

def test_pitmodelx(modelfile, testevent, runts, loopcnt=10, model='oracle'):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    if runts == 'RANK':
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                      pitmodel=modelfile, model=model)        
    else:
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                        'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model)

    #print(np.mean(acc, axis=0))
    return acc, ret

def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    try:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    except:
        acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [3]:
acc, ret, pret = {}, {}, {}

In [4]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for year in ['2018','2019']:
    for runts in ['TIMEDIFF','RANK']:
        for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.73}, mae={3.11}, rmse={4.79},r2={-0.24}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: acc={0.69}, mae={3.19}, rmse={4.87},r2={-0.29}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: acc={0.70}, mae={3.17}, rmse={4.84},r2={-0.19}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
[[ 0.70723104  3.15343915  4.83252122 -0.24008215]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.68}, mae={3.39}, rmse={5.10},r2={-0.29}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
model: acc={0.67}, mae={3.40}, rmse={5.02},r2={-0.34}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
model: acc={0.69}, mae={3.31}, rmse={4.97},r2={-0.40}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
[[ 0.67730496  3.36879433  5.02908453 -0.34263119]
 [ 0.10638298  4.2393617   5.69107177  0.53333692]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=8

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=8

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=8

In [5]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for year in ['2018','2019']:
    for runts in ['TIMEDIFF','RANK']:
        for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.55}, mae={4.01}, rmse={5.79},r2={-1.30}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: acc={0.58}, mae={3.80}, rmse={5.65},r2={-0.97}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
model: acc={0.59}, mae={3.83}, rmse={5.76},r2={-1.10}, 
            naive: acc={0.10}, mae={4.10}, rmse={5.50},r2={0.57}
[[ 0.57319224  3.88007055  5.73427669 -1.12056705]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.55}, mae={4.11}, rmse={5.89},r2={-0.93}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
model: acc={0.59}, mae={4.07}, rmse={5.70},r2={-0.87}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
model: acc={0.57}, mae={4.11}, rmse={5.86},r2={-1.26}, 
            naive: acc={0.11}, mae={4.24}, rmse={5.69},r2={0.53}
[[ 0.56914894  4.09751773  5.81495358 -1.02092739]
 [ 0.10638298  4.2393617   5.69107177  0.53333692]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4


In [6]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[ 0.73015873,  3.10582011,  4.78810254, -0.24021232],
         [ 0.1005291 ,  4.1005291 ,  5.50276506,  0.57198262]],
 
        [[ 0.68783069,  3.18518519,  4.87407568, -0.2891362 ],
         [ 0.1005291 ,  4.1005291 ,  5.50276506,  0.57198262]],
 
        [[ 0.7037037 ,  3.16931217,  4.83538544, -0.19089792],
         [ 0.1005291 ,  4.1005291 ,  5.50276506,  0.57198262]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[ 0.67553191,  3.39361702,  5.09693274, -0.29255276],
         [ 0.10638298,  4.2393617 ,  5.69107177,  0.53333692]],
 
        [[ 0.67021277,  3.39893617,  5.01752249, -0.34025196],
         [ 0.10638298,  4.2393617 ,  5.69107177,  0.53333692]],
 
        [[ 0.68617021,  3.31382979,  4.97279835, -0.39508884],
         [ 0.10638298,  4.2393617 ,  5.69107177,  0.53333692]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[ 0.6984127 ,  3.21693122,  4.92268263, -0.34985685],
   

In [7]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print(np.mean(acc['oracle-TIMEDIFF-2018-inlap-nopitage'], axis=0))
print(np.mean(acc['oracle-TIMEDIFF-2019-inlap-nopitage'], axis=0))

In [ ]:
print(np.mean(acc['pitmodel-TIMEDIFF-2018-inlap-nopitage'], axis=0))
print(np.mean(acc['pitmodel-TIMEDIFF-2019-inlap-nopitage'], axis=0))

In [ ]:
df1 =ret['oracle-RANK-2019-noinlap-nopitage'][0]
df2 =ret['pitmodel-RANK-2019-noinlap-nopitage'][0]
print('2019: oracle:%d, pitmodel:%d'%(len(df1),len(df2)))

In [19]:
def get_best(pitmodel=True, verbose=False):
    best_acc = {2018:[0, ''], 2019:[0,'']}
    best_mae = {2018:[100, ''], 2019:[100,'']}
    for model in ret.keys():
        if model.find('pitmodel')==0:
            if len(ret[model]) > 0:
                oracle = model.replace('pitmodel','oracle')
                if verbose:
                    print('model:',(oracle))
                acc_oracle = np.mean(acc[oracle], axis=0)
                if verbose:
                    print(acc_oracle)

                if verbose:
                    print('model:',(model))
                acc_model = np.mean(acc[model], axis=0)
                if verbose:
                    print(acc_model)

                if model.find('2018') > 0:
                    year = 2018
                else:
                    year = 2019
                    
                if pitmodel:
                    test_model = acc_model
                    modelname = model
                else:
                    test_model = acc_oracle
                    modelname = oracle
                    
                #mae
                if test_model[0, 0] > best_acc[year][0]:
                    best_acc[year][0] = test_model[0, 0]
                    best_acc[year][1] = modelname

                if test_model[0, 1] < best_mae[year][0]:
                    best_mae[year][0] = test_model[0, 1]
                    best_mae[year][1] = modelname
                
    #print('mae',best_mae)
    #print('acc',best_acc)                
    return best_acc, best_mae
                


In [20]:
get_best(pitmodel=True, verbose=False)

({2018: [0.6208112874779541, 'pitmodel-RANK-2018-noinlap-pitage'],
  2019: [0.5162790697674419, 'pitmodel-TIMEDIFF-2019-noinlap-nopitage']},
 {2018: [3.710758377425044, 'pitmodel-RANK-2018-noinlap-pitage'],
  2019: [4.0060060060060065, 'pitmodel-RANK-2019-inlap-nopitage']})

In [21]:
get_best(pitmodel=False, verbose=False)

({2018: [0.7072310405643739, 'oracle-TIMEDIFF-2018-noinlap-pitage'],
  2019: [0.627906976744186, 'oracle-TIMEDIFF-2019-noinlap-nopitage']},
 {2018: [3.1534391534391535, 'oracle-TIMEDIFF-2018-noinlap-pitage'],
  2019: [3.578294573643411, 'oracle-TIMEDIFF-2019-noinlap-nopitage']})

In [8]:
best_acc = {2018:[0, ''], 2019:[0,'']}
best_mae = {2018:[100, ''], 2019:[100,'']}
for model in ret.keys():
    if model.find('pitmodel')==0:
        if len(ret[model]) > 0:
            oracle = model.replace('pitmodel','oracle')
            print('model:',(oracle))
            acc_oracle = np.mean(acc[oracle], axis=0)
            print(acc_oracle)
            
            print('model:',(model))
            acc_model = np.mean(acc[model], axis=0)
            print(acc_model)
            
            if model.find('2018') > 0:
                year = 2018
            else:
                year = 2019
            
            #mae
            if acc_model[0, 0] > best_acc[year][0]:
                best_acc[year][0] = acc_model[0, 0]
                best_acc[year][1] = model
                
            if acc_model[0, 1] < best_mae[year][0]:
                best_mae[year][0] = acc_model[0, 1]
                best_mae[year][1] = model
                
                
print('mae',best_mae)
print('acc',best_acc)

model: oracle-TIMEDIFF-2018-noinlap-pitage
[[ 0.70723104  3.15343915  4.83252122 -0.24008215]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]
model: pitmodel-TIMEDIFF-2018-noinlap-pitage
[[ 0.57319224  3.88007055  5.73427669 -1.12056705]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]
model: oracle-TIMEDIFF-2018-inlap-pitage
[[ 0.67730496  3.36879433  5.02908453 -0.34263119]
 [ 0.10638298  4.2393617   5.69107177  0.53333692]]
model: pitmodel-TIMEDIFF-2018-inlap-pitage
[[ 0.56914894  4.09751773  5.81495358 -1.02092739]
 [ 0.10638298  4.2393617   5.69107177  0.53333692]]
model: oracle-TIMEDIFF-2018-noinlap-nopitage
[[ 0.68783069  3.26984127  4.97281091 -0.38313951]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]
model: pitmodel-TIMEDIFF-2018-noinlap-nopitage
[[ 0.56613757  3.96825397  5.73999623 -1.14009759]
 [ 0.1005291   4.1005291   5.50276506  0.57198262]]
model: oracle-TIMEDIFF-2018-inlap-nopitage
[[ 0.67730496  3.55141844  5.35897401 -0.38543364]
 [ 0.10638298  4.2393617

### test

In [ ]:
tacc, tret = {}, {}

In [ ]:
mid = 'oracle-2018'
tacc[mid], tret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)


In [ ]:
mid = 'ranknet-2018-debug'
tacc[mid], tret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                   'Indy500-2018', 'TIMEDIFF', loopcnt=1,
                                    inlapmode=1,featuremode =stint.FEATURE_STATUS )


In [ ]:
allpits, pitmat, maxlap = stint.get_pitlaps(verbose=True)
#print(len(allpits), maxlap)
nextpit, maxnext = stint.get_nextpit(pitmat, 30)

print(nextpit, maxnext)

In [ ]:
len(pitmat)

In [ ]:
pitmat

In [23]:
df=ret['pitmodel-RANK-2018-noinlap-pitage'][0]
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,20.0,12.0,-8.0,-1,14.0,-6.0,-1
1,14,29,10.0,1.0,-9.0,-1,7.0,-3.0,-1
2,59,29,32.0,22.0,-10.0,-1,28.0,-4.0,-1
3,3,30,10.0,4.0,-6.0,-1,7.0,-3.0,-1
4,20,30,7.0,0.0,-7.0,-1,0.0,-7.0,-1
...,...,...,...,...,...,...,...,...,...
184,25,195,16.0,14.0,-2.0,-1,13.0,-3.0,-1
185,59,195,21.0,21.0,0.0,0,21.0,0.0,0
186,60,195,17.0,15.0,-2.0,-1,15.0,-2.0,-1
187,19,196,15.0,18.0,3.0,1,17.0,2.0,1


In [24]:
df=ret['oracle-TIMEDIFF-2019-inlap-pitage'][0]
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,10,7,31.0,12.0,-19.0,-1,30.0,-1.0,-1
1,24,7,30.0,23.0,-7.0,-1,31.0,1.0,1
2,98,7,27.0,29.0,2.0,1,31.0,4.0,1
3,22,31,15.0,5.0,-10.0,-1,2.0,-13.0,-1
4,2,33,27.0,18.0,-9.0,-1,14.0,-13.0,-1
...,...,...,...,...,...,...,...,...,...
217,3,184,21.0,17.0,-4.0,-1,21.0,0.0,0
218,24,184,17.0,18.0,1.0,1,19.0,2.0,1
219,48,184,20.0,19.0,-1.0,-1,20.0,0.0,0
220,60,184,18.0,20.0,2.0,1,17.0,-1.0,-1


In [25]:

df=ret['oracle-RANK-2019-noinlap-nopitage'][0]
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,10,7,31.0,12.0,-19.0,-1,31.0,0.0,0
1,24,7,30.0,23.0,-7.0,-1,30.0,0.0,0
2,98,7,27.0,29.0,2.0,1,27.0,0.0,0
3,22,31,15.0,5.0,-10.0,-1,0.0,-15.0,-1
4,2,33,27.0,18.0,-9.0,-1,12.0,-15.0,-1
...,...,...,...,...,...,...,...,...,...
210,3,184,21.0,17.0,-4.0,-1,21.0,0.0,0
211,24,184,17.0,18.0,1.0,1,16.0,-1.0,-1
212,48,184,20.0,19.0,-1.0,-1,20.0,0.0,0
213,60,184,18.0,20.0,2.0,1,18.0,0.0,0


In [28]:
np.mean(acc['oracle-TIMEDIFF-2019-noinlap-pitage'],axis=0)

array([[ 0.61085271,  3.87751938,  5.71170474, -0.13350819],
       [ 0.15813953,  4.25581395,  6.45809389,  0.430447  ]])

In [29]:
np.mean(acc['pitmodel-RANK-2018-noinlap-pitage'],axis=0)

array([[ 0.62081129,  3.71075838,  5.40584204, -1.16668121],
       [ 0.1005291 ,  4.10582011,  5.523399  ,  0.56876669]])

In [33]:
np.mean(acc['pitmodel-TIMEDIFF-2019-noinlap-nopitage'],axis=0)

array([[ 0.51627907,  4.5875969 ,  6.50518216, -0.87430609],
       [ 0.15813953,  4.25581395,  6.45809389,  0.430447  ]])

In [37]:
np.mean(acc['pitmodel-RANK-2019-noinlap-pitage'],axis=0)

array([[ 0.49302326,  4.42325581,  6.37255417, -0.89674152],
       [ 0.15813953,  4.25581395,  6.45809389,  0.430447  ]])

In [36]:
np.mean(acc['pitmodel-RANK-2019-inlap-nopitage'],axis=0)

array([[ 0.45795796,  4.00600601,  6.0306237 , -1.94041352],
       [ 0.18018018,  4.03603604,  6.25292724,  0.46046139]])

In [ ]:
correct = df[df['sign']==df['pred_sign']] 
acc = len(correct)/len(df)
acc

In [ ]:
stint.get_evalret(df)

In [ ]:
ret ={ }
ret['oracle'] = df
ret['curtrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTCURTRACK )

In [ ]:
stint.get_evalret(ret['curtrack'])

In [ ]:
ret['zerotrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTZERO )

In [ ]:
stint.get_evalret(ret['zerotrack'])

### test sim by pred

In [ ]:
def simulation2(datasetid, testevent, taskid, runts, expid, predictionlen, datamode, loopcnt, featuremode = stint.FEATURE_STATUS):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init()
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    return b

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY, 3)

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,3)


In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTCURTRACK,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTZERO,5)

In [ ]:
accret = simulation('indy2013-2018-pitage-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5, featuremode = stint.FEATURE_PITAGE)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

### sim pred

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()

#stint._dataset_id = 'indy2013-2018'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

#stint._dataset_id = 'indy2013-2018-nocarid-context40'
#stint._feature_mode = stint.FEATURE_STATUS
stint._dataset_id = 'indy2013-2018-pitage-nocarid-context40'
stint._feature_mode = stint.FEATURE_PITAGE

stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df2 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
stint.get_evalret(df2)
df3 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df3)

In [ ]:
df2[df2['carno']==12]

In [ ]:
df2

In [ ]:
stint.get_evalret(df2)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
    
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
df[df['carno']==12]

In [ ]:
stint.get_evalret(df)

In [ ]:
acc = []
for i in ret.keys():
    df = ret[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
a = [(1,2,3),(2,3,4)]
np.array(a)

In [ ]:
len(ret)

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

### debug

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

### difference ? oracle .vs. pred

In [ ]:
#df, df2
df

In [ ]:
df2

In [ ]:
oracle = df[['carno','startlap']].values
pred =  df2[['carno','startlap']].values

In [ ]:
a = set(['%s-%s'%(x[0],x[1]) for x in oracle])
b = set(['%s-%s'%(x[0],x[1]) for x in  pred])
a -b

In [ ]:
df[df['carno']==3]

In [ ]:
df2[df2['carno']==3]

In [ ]:
_, data2018, _ = stint.load_data('Indy500-2018')
data2018[data2018['completed_laps']==200]

In [ ]:
topK = data2018[data2018['completed_laps']==200]['car_number'].values
topK

In [ ]:
topK_df = df[df['carno'].isin(topK[:5])]
stint.get_evalret(topK_df)

In [ ]:
len(topK_df)

In [ ]:
topK_df2 = df2[df2['carno'].isin(topK[:5])]
stint.get_evalret(topK_df2)

In [ ]:
len(topK_df2)

### check the dataset

In [ ]:
df = ret['oracle-2018']

In [ ]:
df = df[0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
dftemp = df[['carno','startlap', 'startrank','endrank']].sort_values(['carno','startlap'])
test = dftemp[['carno','startlap', 'startrank','endrank']].values.astype(int)
dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])
dfout.to_csv('test_db_ranknet2.csv')

In [ ]:
df = ret['ranknet-2018'][0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
ret['ranknet-timediff-2019']

In [ ]:
ret.keys()

In [ ]:
df=ret['oracle-2018'][0]
df.to_csv('result-stint')

In [ ]:
df = ret['oracle-TIMEDIFF-2018-noinlap-pitage'][0]
df[df['carno']==12]

In [ ]:
df

In [ ]:
df[df['carno']==12]

In [ ]:
df = ret['oracle-TIMEDIFF-2018-inlap-pitage'][0]
df[df['carno']==12]

In [ ]:
df = ret['pitmodel-TIMEDIFF-2018-outlap-nopitage'][0]
df[df['carno']==12]

In [ ]:
stint._inlap_status = 0

stint.init('oracle')

In [ ]:
accx, retx = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)
df_oracle_2018 = retx[0]

In [ ]:
stint.get_evalret(df_oracle_2018)

In [ ]:
accx2, retx2 = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1)
df_oracle_2019 = retx2[0]

In [ ]:
def neweval(df_refer, mid):
    dfml = df_refer
    dfranknet = ret[mid][0]
    carlist = set(list(dfml['carno'].values))
    dfpits = []
    for car in carlist:
        pitlaps = dfml[dfml['carno']==car].startlap
        dfpits.append(dfranknet[(dfranknet['carno']==car) & (dfranknet['startlap'].isin(pitlaps))])

    df_ranknet = pd.concat(dfpits)
    accret = stint.get_evalret(df_ranknet)
    
    return df_ranknet, accret


def neweval2(df_refer, df2):
    dfml = df_refer
    dfranknet = df2
    carlist = set(list(dfml['carno'].values))
    dfpits = []
    for car in carlist:
        pitlaps = dfml[dfml['carno']==car].startlap
        dfpits.append(dfranknet[(dfranknet['carno']==car) & (dfranknet['startlap'].isin(pitlaps))])

    df_ranknet = pd.concat(dfpits)
    accret = stint.get_evalret(df_ranknet)
    
    return df_ranknet, accret

In [ ]:
dfn, accn = {} ,{}

In [ ]:
mid = 'oracle-TIMEDIFF-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

In [ ]:
mid = 'pitmodel-TIMEDIFF-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

In [ ]:
mid = 'pitmodel-RANK-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

In [ ]:
mid = 'pitmodel-RANK-2018-inlap-nopitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

In [ ]:
mid = 'pitmodel-TIMEDIFF-2018-inlap-nopitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

In [ ]:
mid = 'oracle-TIMEDIFF-2018-inlap-pitage'
dfn[mid][dfn[mid]['carno']==12]

In [ ]:
df_oracle_2018[df_oracle_2018['carno']==12]

In [ ]:
acc2, ret2 = {},{}

In [ ]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'oracle-timediff-2018-inlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1,
                                     inlapmode=1,featuremode = stint.FEATURE_STATUS)

In [ ]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-inlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=1,featuremode = stint.FEATURE_STATUS)

In [ ]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-noinlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=0,featuremode = stint.FEATURE_STATUS)

In [ ]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-noinlap-pitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=0,featuremode = stint.FEATURE_PITAGE)

In [ ]:
df= ret2[mid][0]
df[df['carno']==59]

In [ ]:
df

In [ ]:
neweval2(df, df_oracle_2018)

In [ ]:
neweval2(df_oracle_2018,ret2[mid][0])

In [ ]:
neweval2(df_oracle_2018,df_oracle_2018)

In [ ]:
df_oracle_2018[140:]